In [35]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

from IPython.display import Markdown
from dotenv import load_dotenv

In [7]:
load_dotenv("../app/.env")

True

In [26]:
answer_evaluation_template = """You are a helpful and encouraging interviewer.
You will be specified with a topic, a blooms level, and learning outcome along with a question that the student needs to be tested on as well as the student's response to the question. You need to provide an evaluation based on the student's response. The student's response will be delimited with #### characters.

Question: {question}

####
{input}
####

To solve the problem, do the following
- First, work out your own solution to the problem
- Then, compare your solution to the student's solution. Don’t give any answers or hints. Assess the student on the learning outcome provided using a rating of 0 (Unsatisfactory), 1 (Satisfactory) or 2 (Proficient).
- At the end, give some actionable feedback too.

Use the following format:
Actual solution:
```
concise steps to work out the solution and your solution here
```
{{"Answer": 0/1/2, "feedback": str }}
"""

In [27]:
clarify_question_template = """You are a helpful and encouraging interviewer.
You will be specified with a topic, a blooms level, and learning outcome along with a question that the student needs to be tested on along with a series of interactions between a student and you.

The student has asked a clarifying question that you need to answer. The student's response will be delimited with #### characters

####
{input}
####

Important:
- Make sure to not give hints or answer the question.
- If the student asks for the answer, refrain from answering."""

In [28]:
prompt_infos = [
    {
        "name": "answer_evaluation",
        "description": "Good for providing an evaluation for the answer given by the student to a question",
        "prompt_template": answer_evaluation_template,
    },
    {
        "name": "clarify_question_template",
        "description": "Good for answering clarifying questions",
        "prompt_template": clarify_question_template,
    },
]

In [29]:
# chat_model = ChatOpenAI(temperature=0, model="gpt-4")
llm = OpenAI(temperature=0)

In [43]:
from __future__ import annotations

from typing import Any, Dict, List, Optional

from pydantic import Extra

from langchain.base_language import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate


class ReturnStringChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    output_key: str = "text"  #: :meta private:
    output_string: str

    # class Config:
    #     """Configuration for this pydantic object."""

    #     extra = Extra.forbid
    #     arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return []

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # If you want to log something about this run, you can do so by calling
        # methods on the `run_manager`, as shown below. This will trigger any
        # callbacks that are registered for that event.
        if run_manager:
            run_manager.on_text(f"Just returning back: {self.output_string}")

        return {self.output_key: self.output_string}

    @property
    def _chain_type(self) -> str:
        return "return_string_chain"

In [30]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [40]:
random_chain_name = "irrelevant_to_question"
random_chain_prompt = "{input}"
random_chain_description = "Good when the query is irrelevant to the question"

In [42]:
chain = ReturnStringChain(
    output_key="answer", output_string="Irrelevant", prompt=random_chain_prompt
)

In [44]:
destination_chains[random_chain_name] = chain

In [45]:
destination_chains

{'answer_evaluation': LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='You are a helpful and encouraging interviewer.\nYou will be specified with a topic, a blooms level, and learning outcome along with a question that the student needs to be tested on as well as the student\'s response to the question. You need to provide an evaluation based on the student\'s response. The student\'s response will be delimited with #### characters.\n\n####\n{input}\n####\n\nTo solve the problem, do the following\n- First, work out your own solution to the problem\n- Then, compare your solution to the student\'s solution. Don’t give any answers or hints. Assess the student on the learning outcome provided using a rating of 0 (Unsatisfactory), 1 (Satisfactory) or 2 (Proficient).\n- At the end, give some actionable feedback too.\n\nUse the following format:\nActual so

In [46]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [47]:
RouterOutputParser()

RouterOutputParser(default_destination='DEFAULT', next_inputs_type=<class 'str'>, next_inputs_inner_key='input')

In [36]:
Markdown(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT >>


In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """
Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >> Return a markdown code snippet with a JSON object formatted to look like:

{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts. REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >> {destinations}

<< INPUT >> {{input}}

<< OUTPUT >>
"""